# Lab 3 – Module 4: Testing the Perceptron’s Limits

**Time:** ~5 minutes

---

In Module 3 you got 100 % accuracy on data that could be split by a straight line.  
Now let’s go back to the two patterns that *couldn’t* be split in Module 0 — **XOR** and **Circles** — and see if the perceptron can handle them.

## 1. Setup

Run this cell to recreate the XOR and circle datasets and the perceptron tool.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import FloatSlider, Dropdown, interact

np.random.seed(42)

# Activation functions
def sigmoid(z):
    return 1 / (1 + np.exp(-np.clip(z, -500, 500)))

def relu(z):
    return np.maximum(0, z)

def step_fn(z):
    return (z > 0).astype(float)

# --- XOR dataset ---
n_c = 25
corners = [(-1.5, -1.5), (1.5, -1.5), (-1.5, 1.5), (1.5, 1.5)]
labels  = [0, 1, 1, 0]
X_xor, y_xor = [], []
for (cx, cy), lab in zip(corners, labels):
    X_xor.append(np.random.randn(n_c, 2) * 0.3 + [cx, cy])
    y_xor.append(np.full(n_c, lab))
X_xor = np.vstack(X_xor)
y_xor = np.hstack(y_xor)

# --- Circles dataset ---
n_r = 50
ang_in  = np.random.uniform(0, 2 * np.pi, n_r)
r_in    = np.random.uniform(0.3, 0.8, n_r)
X_in    = np.column_stack([r_in * np.cos(ang_in), r_in * np.sin(ang_in)])
ang_out = np.random.uniform(0, 2 * np.pi, n_r)
r_out   = np.random.uniform(1.5, 2.2, n_r)
X_out   = np.column_stack([r_out * np.cos(ang_out), r_out * np.sin(ang_out)])
X_circ  = np.vstack([X_in, X_out])
y_circ  = np.hstack([np.zeros(n_r), np.ones(n_r)])

hard_datasets = {
    'XOR':     (X_xor, y_xor),
    'Circles': (X_circ, y_circ),
}

print('XOR and Circles datasets ready.')
print('Remember: no straight line could separate either of these in Module 0.')

## 2. Try to Classify These Patterns

Use the same sliders as Module 3. Adjust **w\u2081**, **w\u2082**, **bias**, and **activation**. Try everything you can think of.

**Can you reach 100 %?** Pay attention to the highest accuracy you can get and notice what keeps going wrong.

In [ ]:
def test_limits(dataset_name, w1, w2, b, activation_name):
    X, y_true = hard_datasets[dataset_name]
    act = {'Sigmoid': sigmoid, 'ReLU': relu, 'Step': step_fn}[activation_name]
    threshold = 0.5

    g = np.linspace(-3, 3, 200)
    G1, G2 = np.meshgrid(g, g)
    Z_grid = act(w1 * G1 + w2 * G2 + b)
    pred_grid = (Z_grid > threshold).astype(int)

    z_data = act(w1 * X[:, 0] + w2 * X[:, 1] + b)
    pred_data = (z_data > threshold).astype(int)
    correct = pred_data == y_true
    acc = correct.mean() * 100

    fig, ax = plt.subplots(figsize=(8, 8), dpi=100)
    ax.contourf(G1, G2, pred_grid, levels=[-0.5, 0.5, 1.5],
                colors=['#cce0ff', '#ffcccc'], alpha=0.35)
    ax.contour(G1, G2, pred_grid, levels=[0.5], colors=['green'], linewidths=3)

    ax.scatter(X[correct & (y_true == 0), 0], X[correct & (y_true == 0), 1],
              c='blue', s=80, alpha=0.7, edgecolors='k', lw=1.2, label='Class 0')
    ax.scatter(X[correct & (y_true == 1), 0], X[correct & (y_true == 1), 1],
              c='red',  s=80, alpha=0.7, edgecolors='k', lw=1.2, label='Class 1')
    if not correct.all():
        ax.scatter(X[~correct, 0], X[~correct, 1],
                  c='yellow', s=130, marker='X', edgecolors='red', lw=3, label='Wrong')

    ax.set_title(f'{dataset_name}  |  {activation_name}  |  Accuracy: {acc:.0f}%',
                fontsize=13, fontweight='bold')
    ax.set_xlabel('x\u2081'); ax.set_ylabel('x\u2082')
    ax.legend(fontsize=10, loc='upper left')
    ax.set_xlim(-3, 3); ax.set_ylim(-3, 3)
    ax.set_aspect('equal'); ax.grid(True, alpha=0.2)
    plt.tight_layout(); plt.show()

    if acc > 75:
        print(f'{acc:.0f}% \u2014 better than random, but still not perfect.')
        print('Notice that the boundary is always a single straight line.')
    elif acc > 55:
        print(f'{acc:.0f}% \u2014 barely above random guessing (50%).')
    else:
        print(f'{acc:.0f}% \u2014 about the same as flipping a coin.')

interact(
    test_limits,
    dataset_name=Dropdown(options=['XOR', 'Circles'], value='XOR',
                          description='Dataset:'),
    w1=FloatSlider(min=-3, max=3, step=0.1, value=1.0, description='Weight w\u2081:',
                   continuous_update=False),
    w2=FloatSlider(min=-3, max=3, step=0.1, value=1.0, description='Weight w\u2082:',
                   continuous_update=False),
    b=FloatSlider(min=-5, max=5, step=0.1, value=0.0,  description='Bias b:',
                  continuous_update=False),
    activation_name=Dropdown(options=['Sigmoid', 'ReLU', 'Step'], value='Sigmoid',
                             description='Activation:')
);

## 3. Why the Perceptron Hits a Wall

No matter what you tried, you kept hitting a ceiling around 50–75 %. Here’s why:

A single perceptron can only draw **one straight line**. That’s it.

- **XOR** has four clusters arranged diagonally. Two are blue, two are red. One line can get two clusters right but will always put the other two on the wrong side. You would need **at least two lines** to carve out the diagonal pattern.

- **Circles** has blue dots surrounded by a red ring. A straight line will always slice through both colors. You would need a **curved** (circular) boundary.

Even though activation functions can bend space (Module 1), a single perceptron still only makes **one** decision boundary. To break through, you need **more perceptrons working together** — in other words, a *network*.

## 4. The Whole Lab at a Glance

Look back at the arc you’ve just completed:

| Module | What you learned |
|--------|------------------|
| **0** | Straight lines **fail** on some patterns (XOR, circles). |
| **1** | Activation functions **bend space** so straight rules become curved. |
| **2** | Different activations have different strengths (saturation, smoothness). |
| **3** | A perceptron = weighted sum + activation. It can separate simple patterns. |
| **4** | A *single* perceptron still **hits a wall** on XOR and circles. |

The logical next step? **Connect multiple perceptrons in layers.** Each one draws its own boundary; together they can carve out any shape. That’s a **neural network** — and it’s what you’ll explore in the next lab.

## Answer‑Sheet Questions (Q13 – Q15)

**Q13.** What was your best accuracy on XOR? On the circles? Describe what kept happening each time you tried a new setting — what ceiling did you keep hitting, and why couldn’t you push past it?

**Q14.** A single perceptron can only draw one straight line. How many lines would you actually need to correctly separate XOR’s four corners? Describe or sketch where you would place them.

**Q15.** Look back at the whole arc of this lab: straight lines failed → activation functions bent space → a single perceptron still hit a wall. What is the logical next move? What would you add to the system to finally break through?

---

**Congratulations!** You’ve completed Lab 3. Submit your answer sheet and get ready for Lab 4, where you’ll build a multi-layer neural network that finally solves these impossible problems.